In [4]:
!pip install pretrainedmodels split-folders tqdm

     |████████████████████████████████| 58 kB 7.2 MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=b84ae897f307d17d65683856d04d0bad1a75e028c8de69db9bf6de1987ee2da6
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -q /content/drive/My\ Drive/data.zip

In [5]:
from __future__ import print_function, division
import splitfolders
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from copy import deepcopy
import pretrainedmodels
import pretrainedmodels.utils as utils
from shutil import copyfile,copy
import glob
#os.environ['TORCH_HOME'] = '.'

In [6]:
print("Available model : \n ",pretrainedmodels.model_names)

Available model : 
  ['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [7]:
# We are using resnet50 first

In [8]:
DATA_SOURCE = "data/train"
DATA_PATH = "dataset"
MODEL_NAME = "resnet50"
VAL_RATIO = 0.1
SEED = 1337
EPOCHS = 15
PATIENCE = 5
GAMMA = 0.1
LEARNING_RATE = 0.001
MOMENTUM = 0.1
STEP_SIZE = 7
N_EPOCHS = 50
TRAIN_LOADER_BATCH_SIZE = 32
TEST_LOADER_BATCH_SIZE = 20 
N_WORKER = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # use gpu if available

In [9]:
def prepare_folds(path_in,path_out,val_ratio=0.2):
  splitfolders.ratio(path_in, output=path_out, seed=SEED, ratio=(1-(val_ratio*2),val_ratio,val_ratio), group_prefix=None)

In [10]:
def get_loaders(data_dir,transform,bs=32,t_bs=20,n_worker=1):

  image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),transform) for x in ['train', 'val']}

  dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=bs, shuffle=True, num_workers=n_worker)for x in ['train', 'val']}

  image_tests = {"test": datasets.ImageFolder(os.path.join(data_dir, "test"),transform)}

  dataload_test = {"test": torch.utils.data.DataLoader(image_tests["test"], batch_size=t_bs,shuffle=False, num_workers=n_worker)}

  dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

  class_names = image_datasets['train'].classes

  return dataloaders,dataload_test,dataset_sizes,class_names

In [11]:
def imshow(inp, model, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = model.std * inp + model.mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [12]:
class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10, percentage=False):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)

        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        if np.isnan(metrics):
            return True

        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
            print('improvement!')
        else:
            self.num_bad_epochs += 1
            print(f'no improvement, bad_epochs counter: {self.num_bad_epochs}')

        if self.num_bad_epochs >= self.patience:
            return True

        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - (
                            best * min_delta / 100)
            if mode == 'max':
                self.is_better = lambda a, best: a > best + (
                            best * min_delta / 100)

In [13]:
def train_model(model,dataloaders,dataset_sizes, criterion, optimizer, scheduler, num_epochs=25, patience=5):
    
    es = EarlyStopping(patience=patience)
    terminate_training = False
    
    since = time.time()

    best_model_wts = deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = deepcopy(model.state_dict())
                
            if phase == 'val' and es.step(epoch_loss):
                terminate_training = True
                print('early stop criterion is met, we can stop now')
                break
    
        print()
        if terminate_training:
            break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [14]:
#source kaggle
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [15]:
def visualize_model_test(model, num_images=20):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataload_test['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            print(preds)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [16]:
def __train__(model_name='xception'):
  model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
  model.eval()
  transform_img = utils.TransformImage(model)
  dataloaders, dataload_test, dataset_sizes, class_names = get_loaders(
      DATA_PATH,transform_img,bs=TRAIN_LOADER_BATCH_SIZE,t_bs=TEST_LOADER_BATCH_SIZE,n_worker=N_WORKER)
  num_ftrs = model.last_linear.in_features
  model.last_linear = nn.Linear(num_ftrs, len(class_names))
  model = model.to(device)
  criterion = nn.CrossEntropyLoss()
  # Observe that all parameters are being optimized
  optimizer_ft = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
  # Decay LR by a factor of GAMMA every STEP_SIZE epochs
  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=STEP_SIZE, gamma=GAMMA)
  model = train_model(model,dataloaders,dataset_sizes, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=N_EPOCHS)
  return model

In [135]:
#!rm -rf dataset; rm -rf data

In [17]:
prepare_folds(DATA_SOURCE,DATA_PATH,VAL_RATIO)

Copying files: 847 files [00:00, 7594.77 files/s]


In [18]:
model = __train__(MODEL_NAME)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Epoch 0/49
----------
train Loss: 0.6921 Acc: 0.5769
val Loss: 0.6859 Acc: 0.5476

Epoch 1/49
----------
train Loss: 0.6530 Acc: 0.6228
val Loss: 0.6643 Acc: 0.5833
improvement!

Epoch 2/49
----------
train Loss: 0.6248 Acc: 0.6746
val Loss: 0.6484 Acc: 0.5952
improvement!

Epoch 3/49
----------
train Loss: 0.6036 Acc: 0.6834
val Loss: 0.6370 Acc: 0.6190
improvement!

Epoch 4/49
----------
train Loss: 0.5677 Acc: 0.7663
val Loss: 0.6342 Acc: 0.5952
improvement!

Epoch 5/49
----------
train Loss: 0.5420 Acc: 0.7633
val Loss: 0.6216 Acc: 0.6429
improvement!

Epoch 6/49
----------
train Loss: 0.5220 Acc: 0.7751
val Loss: 0.6124 Acc: 0.6548
improvement!

Epoch 7/49
----------
train Loss: 0.5145 Acc: 0.8107
val Loss: 0.6133 Acc: 0.6667
no improvement, bad_epochs counter: 1

Epoch 8/49
----------
train Loss: 0.5002 Acc: 0.8299
val Loss: 0.6058 Acc: 0.6429
improvement!

Epoch 9/49
----------
train Loss: 0.4961 Acc: 0.8299
val Loss: 0.6074 Acc: 0.6190
no improvement, bad_epochs counter: 1

Epo

In [19]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [20]:
torch.save(model.state_dict(), 'saved_model_'+MODEL_NAME+'_.pth')

In [21]:
transform_img = utils.TransformImage(model)
dataloaders, dataload_test, dataset_sizes, class_names = get_loaders(
      DATA_PATH,transform_img,bs=TRAIN_LOADER_BATCH_SIZE,t_bs=TEST_LOADER_BATCH_SIZE,n_worker=N_WORKER)